# File reading and data cleaning

In [1]:
import pandas as pd
import numpy as np
import re
df = pd.read_excel(r"C:\Users\Vanessa\Downloads\Dados\Programas\Repositório Git\NGO_DataAnalysis\excel_exemplo.xls", usecols = [0, 1, 2, 4, 5, 7], index_col = None)

In [2]:
# add "month" and "year" columns
month = df.iloc[0,1].month
year = df.iloc[0,1].year
data = str(year) + '-' + str(month) + '-' + '01'
df['data'] = pd.to_datetime(data)
df.drop(df.index[0:3], inplace = True)

In [3]:
# reorder columns and drop unwanted rows 
df.columns = ['dia', 'receita', 'depositante', 'despesas', 'favorecido', 'saldo', 'data']
df = df[['receita', 'depositante', 'despesas', 'favorecido', 'data']]

# Categories classification

In [5]:
# create "expenses" dataframe and drop nulls
expenses = df[['despesas', 'favorecido', 'data']].copy()
expenses.dropna(subset = 'despesas', axis = 0, inplace=True)
expenses.reset_index(drop = True, inplace = True)
expenses

,despesas,favorecido,data
0,363.00,Ração - Agropecuaria 101,2016-09-01
1,10.00,Combust.Posto S. Cristovão,2016-09-01
2,143.57,Combust.Posto Shark,2016-09-01
3,50.00,Saque no TAA,2016-09-01
4,"4,260.00",Salário - Ieda Maria + Aliment,2016-09-01
5,80.00,Combust.Auto PostoN.Camp.,2016-09-01
6,12.90,Dupão - Alimentação,2016-09-01
7,40.00,Saque no TAA,2016-09-01
8,63.00,Tarifa bancária,2016-09-01
9,269.00,Ração - Alexandre Pessoa,2016-09-01


In [6]:
# create "income" dataframe and drop nulls
income = df[['receita', 'depositante', 'data']].copy()
income.dropna(subset = 'receita', axis = 0, inplace=True)
income.reset_index(drop = True, inplace = True)
income

,receita,depositante,data
0,350.00,Doações,2016-09-01
1,360.00,Doações,2016-09-01
2,140.00,Doações,2016-09-01
3,440.00,Doações,2016-09-01
4,120.00,NaN,2016-09-01
5,510.00,Doações,2016-09-01
6,10.00,Doações,2016-09-01
7,70.00,Doações,2016-09-01
8,100.00,Doações,2016-09-01
9,230.00,Doações,2016-09-01


In [8]:
# create categories for income

mask = income['depositante'].str.contains('doaç|ração|v.d.|\d|DD|cielo|cielo|pag|pic', regex = True, flags = re.I)
doa = np.where(mask, 'doações e vendas', income['depositante'])
income.loc[:,'depositante'] = doa

mask2 = income['depositante'].str.contains('empr|apli|resg|rend', regex = True, flags = re.I)
emp = np.where(mask2, 'empréstimos e aplicações', income['depositante'])
income.loc[:,'depositante'] = emp

mask3 = income['depositante'].str.contains('Pista', regex = True, flags = re.I)
proj = np.where(mask3, 'projeto Autopista Litoral Sul', income['depositante'])
income.loc[:,'depositante'] = proj

mask4 = income['depositante'].str.contains('justiça|PMF ', regex = True, flags = re.I)
gov = np.where(mask4, 'governo', income['depositante'])
income.loc[:,'depositante'] = gov

mask5 = income['depositante'].str.contains('div|tran|int|seguro|rog|BB|dif', regex = True, flags = re.I)
out = np.where(mask5, 'outros', income['depositante'])
income.loc[:,'depositante'] = out

In [7]:
# create categories for 'expenses'

mask = expenses['favorecido'].str.contains('sal|intercamb|alim|lanch|temperinho|rest|pão|café|panif|doce|confeitaria|subway|gratificação|sabor|hong ju|mansa', flags = re.I)
sal = np.where(mask, 'salários e auxílios alimentação', expenses['favorecido'])
expenses.loc[:,'favorecido'] = sal

mask2 = expenses['favorecido'].str.contains('sup|super|hiper|hipper|mercado|bistek|milium|embala|atacad|cassol|agua|distrib|loja|zeus|comerc', flags = re.I)
sup = np.where(mask2, 'suprimentos', expenses['favorecido'])
expenses.loc[:,'favorecido'] = sup

mask3 = expenses['favorecido'].str.contains('medic|clínica|clinica|vet|remédio|farm|biofilia|gral|asamed|pet|cuidados|panvel|acupuntura|raio x|praiana|castração|tosa|cremat', flags = re.I)
medic = np.where(mask3, 'medicações e veterinário', expenses['favorecido'])
expenses.loc[:,'favorecido'] = medic

mask4 = expenses['favorecido'].str.contains('hosp|poeta|lilian ribeiro|lar|cuidadora|casa|canil|canis|inse', flags = re.I)
hosp = np.where(mask4, 'hospedagem e manutenção de canis', expenses['favorecido'])
expenses.loc[:,'favorecido'] = hosp

mask5 = expenses['favorecido'].str.contains('agro|ração|reção|pecuária|terra|arcadia', flags = re.I)
rac = np.where(mask5, 'ração e agropecuária', expenses['favorecido'])
expenses.loc[:,'favorecido'] = rac

mask7 = expenses['favorecido'].str.contains('combust|escap|posto|seguro|veíc|motorista|car|borrack|park|auto|taxi|táxi|tàxi|frete|motor|estac|floripeças|multa|mecanica|mecânica|mecãnica|parachoque|borrach|estrela|localiza', flags = re.I)
trans = np.where(mask7, 'transporte', expenses['favorecido'])
expenses.loc[:,'favorecido'] = trans

mask6 = expenses['favorecido'].str.contains('camis|malhas|mat|jebelus|agenda|vest|patchwork|plumas|bazar|graf|gráf|digital|mtools|eletr|hd|site|notebook|face|cópias|copia|plot|mídia|chip|marca|mix|print', flags = re.I)
vend = np.where(mask6, 'bazar, divulgação e tecnologias', expenses['favorecido'])
expenses.loc[:,'favorecido'] = vend

mask8 = expenses['favorecido'].str.contains('empr|apli|renda', flags = re.I)
emp = np.where(mask8, 'empréstimos e aplicações', expenses['favorecido'])
expenses.loc[:,'favorecido'] = emp

mask9 = expenses['favorecido'].str.contains('tarif|INSS|IR|taxa|IPVA|DARF|celesc|tim|distrit|alvará|cadastro|licen|trib|certif|dss|cmf', flags = re.I)
tax = np.where(mask9, 'taxas, tarifas e impostos', expenses['favorecido'])
expenses.loc[:,'favorecido'] = tax

outros_mask = ~(expenses['favorecido'].isin(['salários e auxílios alimentação', 'suprimentos', 'medicações e veterinário', 'hospedagem e manutenção de canis', 'ração e agropecuária', 'bazar, divulgação e tecnologias', 'transporte', 'empréstimos e aplicações', 'taxas, tarifas e impostos']))
expenses.loc[outros_mask,'favorecido'] = "outros"

In [9]:
#converting data types
expenses['despesas'] = expenses['despesas'].astype(str).str.strip()
expenses['despesas'] = expenses['despesas'].str.replace(',', '')
expenses['despesas'] = expenses['despesas'].astype(float)
income['receita'] = income['receita'].astype(str).str.strip()
income['receita'] = income['receita'].str.replace(',', '')
income['receita'] = income['receita'].astype(float)

In [10]:
expenses['favorecido'].value_counts()

outros                              13
ração e agropecuária                11
transporte                           9
taxas, tarifas e impostos            3
salários e auxílios alimentação      2
medicações e veterinário             2
hospedagem e manutenção de canis     2
bazar, divulgação e tecnologias      1
Name: favorecido, dtype: int64

In [11]:
income['depositante'].value_counts()

doações e vendas                 20
projeto Autopista Litoral Sul     1
empréstimos e aplicações          1
Name: depositante, dtype: int64

# Full script

In [ ]:
# add "month" and "year" columns
month = df.iloc[0,1].month
year = df.iloc[0,1].year
data = str(year) + '-' + str(month) + '-' + '01'
df['data'] = pd.to_datetime(data)
df.drop(df.index[0:3], inplace = True)

# reorder columns and drop unwanted rows 
df.columns = ['dia', 'receita', 'depositante', 'despesas', 'favorecido', 'saldo', 'data']
df = df[['receita', 'depositante', 'despesas', 'favorecido', 'data']]

# create "expenses" dataframe and drop nulls
expenses = df[['despesas', 'favorecido', 'data']].copy()
expenses.dropna(subset = 'despesas', axis = 0, inplace=True)
expenses.reset_index(drop = True, inplace = True)

# create "income" dataframe and drop nulls
income = df[['receita', 'depositante', 'data']].copy()
income.dropna(subset = 'receita', axis = 0, inplace=True)
income.reset_index(drop = True, inplace = True)

# create categories for 'expenses'

mask = expenses['favorecido'].str.contains('sal|intercamb|alim|lanch|temperinho|rest|pão|café|panif|doce|confeitaria|subway|gratificação|sabor|hong ju|mansa', flags = re.I)
sal = np.where(mask, 'salários e auxílios alimentação', expenses['favorecido'])
expenses.loc[:,'favorecido'] = sal

mask2 = expenses['favorecido'].str.contains('sup|super|hiper|hipper|mercado|bistek|milium|embala|atacad|cassol|agua|distrib|loja|zeus|comerc', flags = re.I)
sup = np.where(mask2, 'suprimentos', expenses['favorecido'])
expenses.loc[:,'favorecido'] = sup

mask3 = expenses['favorecido'].str.contains('medic|clínica|clinica|vet|remédio|farm|biofilia|gral|asamed|pet|cuidados|panvel|acupuntura|raio x|praiana|castração|tosa|cremat', flags = re.I)
medic = np.where(mask3, 'medicações e veterinário', expenses['favorecido'])
expenses.loc[:,'favorecido'] = medic

mask4 = expenses['favorecido'].str.contains('hosp|poeta|lilian ribeiro|lar|cuidadora|casa|canil|canis|inse', flags = re.I)
hosp = np.where(mask4, 'hospedagem e manutenção de canis', expenses['favorecido'])
expenses.loc[:,'favorecido'] = hosp

mask5 = expenses['favorecido'].str.contains('agro|ração|reção|pecuária|terra|arcadia', flags = re.I)
rac = np.where(mask5, 'ração e agropecuária', expenses['favorecido'])
expenses.loc[:,'favorecido'] = rac

mask7 = expenses['favorecido'].str.contains('combust|escap|posto|seguro|veíc|motorista|car|borrack|park|auto|taxi|táxi|tàxi|frete|motor|estac|floripeças|multa|mecanica|mecânica|mecãnica|parachoque|borrach|estrela|localiza', flags = re.I)
trans = np.where(mask7, 'transporte', expenses['favorecido'])
expenses.loc[:,'favorecido'] = trans

mask6 = expenses['favorecido'].str.contains('camis|malhas|mat|jebelus|agenda|vest|patchwork|plumas|bazar|graf|gráf|digital|mtools|eletr|hd|site|notebook|face|cópias|copia|plot|mídia|chip|marca|mix|print', flags = re.I)
vend = np.where(mask6, 'bazar, divulgação e tecnologias', expenses['favorecido'])
expenses.loc[:,'favorecido'] = vend

mask8 = expenses['favorecido'].str.contains('empr|apli|renda', flags = re.I)
emp = np.where(mask8, 'empréstimos e aplicações', expenses['favorecido'])
expenses.loc[:,'favorecido'] = emp

mask9 = expenses['favorecido'].str.contains('tarif|INSS|IR|taxa|IPVA|DARF|celesc|tim|distrit|alvará|cadastro|licen|trib|certif|dss|cmf', flags = re.I)
tax = np.where(mask9, 'taxas, tarifas e impostos', expenses['favorecido'])
expenses.loc[:,'favorecido'] = tax

outros_mask = ~(expenses['favorecido'].isin(['salários e auxílios alimentação', 'suprimentos', 'medicações e veterinário', 'hospedagem e manutenção de canis', 'ração e agropecuária', 'bazar, divulgação e tecnologias', 'transporte', 'empréstimos e aplicações', 'taxas, tarifas e impostos']))
expenses.loc[outros_mask,'favorecido'] = "outros"

# create categories for income

mask = income['depositante'].str.contains('doaç|ração|v.d.|\d|DD|cielo|cielo|pag|pic', regex = True, flags = re.I)
doa = np.where(mask, 'doações e vendas', income['depositante'])
income.loc[:,'depositante'] = doa

mask2 = income['depositante'].str.contains('empr|apli|resg|rend', regex = True, flags = re.I)
emp = np.where(mask2, 'empréstimos e aplicações', income['depositante'])
income.loc[:,'depositante'] = emp

mask3 = income['depositante'].str.contains('Pista', regex = True, flags = re.I)
proj = np.where(mask3, 'projeto Autopista Litoral Sul', income['depositante'])
income.loc[:,'depositante'] = proj

mask4 = income['depositante'].str.contains('justiça|PMF ', regex = True, flags = re.I)
gov = np.where(mask4, 'governo', income['depositante'])
income.loc[:,'depositante'] = gov

mask5 = income['depositante'].str.contains('div|tran|int|seguro|rog|BB|dif', regex = True, flags = re.I)
out = np.where(mask5, 'outros', income['depositante'])
income.loc[:,'depositante'] = out

#converting data types
expenses['despesas'] = expenses['despesas'].astype(str).str.strip()
expenses['despesas'] = expenses['despesas'].str.replace(',', '')
expenses['despesas'] = expenses['despesas'].astype(float)
income['receita'] = income['receita'].astype(str).str.strip()
income['receita'] = income['receita'].str.replace(',', '')
income['receita'] = income['receita'].astype(float)